In [17]:
import logging
import pyodbc
import pandas as pd
from datetime import datetime

mdb_file = r"C:\Users\gia.luongdo\Desktop\ERP-Importer\db_Artikel_Export2.mdb"
sku_basis_csv = r"C:\Users\gia.luongdo\Desktop\ERP-Importer\IMPORTER_SKU_Neuanlage_Basis.csv"
conn_str = (
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    f"DBQ={mdb_file};"
)
# Configure logging to write to a file
logging.basicConfig(
    filename='importer_log.txt',
    filemode='a',
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO)
# Connect to the Access database
conn = pyodbc.connect(conn_str)
logging.info("Connected to the database successfully.")
#IMPORTER_ARTICLE_Neuanlage_Basis:
#Reads data from the database
def read_and_write_aku_data():
    table_name = 't_Art_Mega_SKU'
    query = f"SELECT sku.ArtikelCode as SKU, m.Ursprungsland, t.ArtBem as name FROM {table_name} sku inner join t_Art_Text_DE t on sku.ArtNr = t.ArtNr inner join t_Art_MegaBase m on sku.ArtNR = m.ArtNr WHERE Marke IN ('Corporate', 'EXCD', 'XO')"
    df = pd.read_sql(query, conn)

    # Add columns with specified values
    df['company'] = 0
    df['automatic_batch_numbering_pattern'] = '{No,000000000}'
    df['batch_management'] = 2
    df['batch_number_range'] = 'Chargen'
    df['batch_numbering_type'] = 3
    df['date_requirement'] = 1
    df['discountable'] = 'ja'
    df['factory'] = 'Düsseldorf'
    df['isPi'] = 'ja'
    df['isShopArticle'] = 'ja'
    df['isSl'] = 'ja'
    df['isSt'] = 'ja'
    df['isVerifiedArticle'] = 'ja'
    df['isCatalogArticle'] = 'ja'
    df['unitPi'] = 'Stk'
    df['unitSl'] = 'Stk'
    df['unitSt'] = 'Stk'
    #df['name'] = df['name']
    df['replacement_time'] = 1
    df['taxPi'] = 'Waren'
    df['taxSl'] = 'Waren'
    df['valid_from'] = datetime.now().strftime("%Y%m%d")

    # Take only first 2 characters from Ursprungsland and rename to 'country', reorder columns
    df['country_of_origin'] = df['country_of_origin'].str[:2]
    df = df[['SKU', 'company', 'country_of_origin', 'automatic_batch_numbering_pattern', 'batch_management', 'batch_number_range', 'batch_numbering_type', 'date_requirement', 'discountable', 'factory', 'isPi', 'isShopArticle', 'isSl', 'isSt', 'isVerifiedArticle', 'isCatalogArticle', 'unitPi', 'unitSl', 'unitSt', 'name', 'replacement_time', 'taxPi', 'taxSl', 'valid_from']]

    # Write DataFrame to CSV with correct separator and columns
    df.to_csv(sku_basis_csv, index=False, encoding='windows-1252', sep=',')
    print(f'Data exported to {sku_basis_csv}')
    logging.info(f'Data exported to {sku_basis_csv}')


# Call the function to execute the logic
try:
    read_and_write_aku_data()
    #read_and_write_classification_data()
    #read_and_write_Zuordnung_Basis()
    #read_and_write_Schlüsselworte_Basis()
except Exception as e:
    print(f"An error occurred: {e}")
    logging.error(f"An error occurred: {e}")




conn.close()

An error occurred: Execution failed on sql 'SELECT sku.ArtikelCode as SKU, m.Ursprungsland, t.ArtBem as name FROM t_Art_Mega_SKU sku inner join t_Art_Text_DE t on sku.ArtNr = t.ArtNr inner join t_Art_MegaBase m on sku.ArtNR = m.ArtNr WHERE Marke IN ('Corporate', 'EXCD', 'XO')': ('42000', "[42000] [Microsoft][ODBC-Treiber für Microsoft Access] Syntaxfehler (fehlender Operator) in Abfrageausdruck 'sku.ArtNr = t.ArtNr inner join t_Art_MegaBase m on sku.ArtNR = m.ArtN'. (-3100) (SQLExecDirectW)")


C:\Users\gia.luongdo\AppData\Local\Temp\ipykernel_16080\3951950549.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
